# CAMO-Net ETL – BRONZE → SILVER
### Notebook de inspeção passo a passo

Este notebook reproduz o funcionamento do script `02_bronze_to_silver.py`:

1. Leitura dos arquivos Parquet na camada bronze  
2. Padronização de nomes de colunas (snake_case)  
3. Identificação e limpeza de colunas de medicamentos  
4. Identificação e tratamento de colunas PII (anonimização/remoção)  
5. Escrita dos dados na camada silver  
6. Geração de um resumo da execução


In [1]:
from pathlib import Path
import pandas as pd
import re
import hashlib

bronze_path = bronze_path = Path('../../data/bronze/')
silver_path = silver_path = Path('../../data/silver/')

In [2]:
print("Bronze path:", bronze_path)
print("Existe?", bronze_path.exists())

if not bronze_path.exists():
    raise FileNotFoundError("A pasta BRONZE não existe. Ajuste o caminho do `bronze_path`.")

parquet_files = sorted(bronze_path.glob('*.parquet'))

print(f"\nTotal de arquivos Parquet encontrados em bronze: {len(parquet_files)}\n")
for i, f in enumerate(parquet_files, 1):
    print(f"{i:2d}. {f.name}")


Bronze path: ../../data/bronze
Existe? True

Total de arquivos Parquet encontrados em bronze: 9

 1. TAB_ATENDIMENTO.parquet
 2. TAB_ATENDIMENTO_ANALISE.parquet
 3. TAB_ATEND_DIAG.parquet
 4. TAB_CIAP_DIAGNOSTICO.parquet
 5. TAB_CID_DIAGNOSTICO.parquet
 6. TAB_MEDICAMENTO.parquet
 7. TAB_MEDPRESCRITO_ANALISE.parquet
 8. TAB_MED_PRESCRITO.parquet
 9. TAB_UNIDADE_SAUDE.parquet


In [3]:
def to_snake_case(name):
    """
    Converte string para snake_case.
    Mesma lógica do script bronze_to_silver.
    """
    # Remover caracteres especiais
    name = re.sub(r'[^\w\s]', '', name)
    # Substituir espaços por underscore
    name = re.sub(r'\s+', '_', name)
    # Converter para minúsculas
    name = name.lower()
    # Remover underscores duplicados
    name = re.sub(r'_+', '_', name)
    # Remover underscores no início e fim
    name = name.strip('_')
    return name


In [4]:
def hash_pii(value):
    """
    Aplica hash SHA256 em dados sensíveis.
    """
    if pd.isna(value):
        return None
    value_str = str(value)
    return hashlib.sha256(value_str.encode()).hexdigest()


In [5]:
def clean_medication_name(name):
    """
    Limpa e padroniza nomes de medicamentos:
    - strip
    - maiúsculas
    - remove espaços duplicados
    """
    if pd.isna(name):
        return None
    
    cleaned = str(name).strip().upper()
    cleaned = re.sub(r'\s+', ' ', cleaned)
    return cleaned


In [6]:
def identify_pii_columns(df):
    """
    Identifica colunas que podem conter dados sensíveis de PACIENTE.
    
    A lógica foi restringida para evitar anonimizar, por exemplo,
    'nome_unidade', 'nome_ubs', etc.
    """
    # Campos genéricos sensíveis (normalmente ligados a paciente ou responsável)
    pii_keywords_base = [
        'cpf', 'rg', 'cns', 'telefone', 'celular', 'email'
    ]
    
    # Campos explicitamente ligados a paciente (ou mãe do paciente)
    pii_keywords_paciente = [
        'nome_paciente',
        'paciente_nome',
        'nome_mae',
        'mae_paciente',
        'nome_social',
        'endereco_paciente',
        'logradouro_paciente',
        'rua_paciente'
    ]
    
    pii_columns = []
    
    for col in df.columns:
        col_lower = col.lower()
        
        # 1) Padrões explícitos de paciente (já em snake_case)
        if any(p in col_lower for p in pii_keywords_paciente):
            pii_columns.append(col)
            continue
        
        # 2) Campos base (cpf, rg, cns, telefone, etc.)
        #    MAS evitando nomes de unidade, serviço, estabelecimento etc.
        if any(k in col_lower for k in pii_keywords_base):
            if not any(bad in col_lower for bad in ['unidade', 'ubs', 'estabelecimento', 'servico', 'serviço']):
                pii_columns.append(col)
    
    return pii_columns

def identify_medication_columns(df):
    """
    Identifica colunas que contêm nomes de medicamentos.
    """
    med_keywords = [
        'medicamento', 'med', 'principio', 'substancia', 
        'nome_med', 'descricao_med'
    ]
    
    med_columns = []
    
    for col in df.columns:
        col_lower = col.lower()
        if any(keyword in col_lower for keyword in med_keywords):
            if 'id' not in col_lower or 'nome' in col_lower or 'descricao' in col_lower:
                med_columns.append(col)
    
    return med_columns


In [8]:
def process_bronze_to_silver(bronze_path, silver_path):
    """
    Replica a lógica do script 02_bronze_to_silver.py para todos os Parquets.
    """
    print("="*80)
    print("CAMO-Net ETL Pipeline - BRONZE → SILVER")
    print("="*80)
    
    silver_path.mkdir(parents=True, exist_ok=True)
    
    parquet_files = list(bronze_path.glob('*.parquet'))
    if not parquet_files:
        print(f"\n⚠️  Nenhum arquivo Parquet encontrado em {bronze_path}")
        return []
    
    print(f"\nArquivos encontrados: {len(parquet_files)}\n")
    
    results = []
    
    for parquet_file in parquet_files:
        table_name = parquet_file.stem
        print("\n" + "─"*80)
        print("Processando:", table_name)
        print("─"*80)
        
        try:
            df = pd.read_parquet(parquet_file)
            original_rows = len(df)
            original_cols = len(df.columns)
            
            print(f"  • Linhas: {original_rows:,}")
            print(f"  • Colunas originais: {original_cols}")
            
            # 1. Padronizar nomes das colunas
            print("\n  [1] Padronizando nomes das colunas...")
            old_columns = df.columns.tolist()
            new_columns = [to_snake_case(col) for col in old_columns]
            df.columns = new_columns
            renamed_count = sum(1 for o, n in zip(old_columns, new_columns) if o != n)
            print(f"      ✓ {renamed_count} colunas renomeadas para snake_case")
            
            # 2. Limpar nomes de medicamentos
            print("\n  [2] Limpando nomes de medicamentos...")
            med_columns = identify_medication_columns(df)
            if med_columns:
                print(f"      • Colunas identificadas: {med_columns}")
                for col in med_columns:
                    df[col] = df[col].apply(clean_medication_name)
                print(f"      ✓ {len(med_columns)} colunas de medicamentos padronizadas")
            else:
                print("      • Nenhuma coluna de medicamento identificada")
            
            # 3. Anonimizar dados sensíveis (PII)
            print(f"\n  [3] Anonimizando dados sensíveis (PII)...")
            pii_columns = identify_pii_columns(df)

            if pii_columns:
                print(f"      • Colunas PII identificadas: {pii_columns}")
                
                for col in pii_columns:
                    col_lower = col.lower()
                    
                    # Remover nomes de paciente / mãe (não queremos guardar nem hash)
                    if any(p in col_lower for p in ['nome_paciente', 'paciente_nome', 'nome_mae', 'nome_social']):
                        print(f"      ⚠️  Removendo coluna de nome de paciente: {col}")
                        df = df.drop(columns=[col])
                    
                    # Remover CPF do paciente
                    elif 'cpf' in col_lower:
                        print(f"      ⚠️  Removendo coluna de CPF: {col}")
                        df = df.drop(columns=[col])
                    
                    # Demais dados sensíveis (telefone, email, etc.): aplicar hash
                    else:
                        print(f"      🔒 Aplicando hash em: {col}")
                        df[col] = df[col].apply(hash_pii)
                
                print(f"      ✓ {len(pii_columns)} colunas PII tratadas")
            else:
                print(f"      • Nenhuma coluna PII identificada")
            
            # 4. Salvar em Silver
            output_file = silver_path / f"{table_name}.parquet"
            df.to_parquet(output_file, engine='pyarrow', compression='snappy', index=False)
            
            final_cols = len(df.columns)
            file_size = output_file.stat().st_size / (1024 * 1024)
            
            print(f"\n  ✓ Salvo em: {output_file.name}")
            print(f"  • Colunas finais: {final_cols}")
            print(f"  • Tamanho: {file_size:.2f} MB")
            
            results.append({
                'tabela': table_name,
                'status': 'SUCCESS',
                'linhas': original_rows,
                'colunas_antes': original_cols,
                'colunas_depois': final_cols,
                'tamanho_mb': file_size
            })
        
        except Exception as e:
            print("  ✗ ERRO:", e)
            results.append({
                'tabela': table_name,
                'status': 'FAILED',
                'erro': str(e)
            })
    
    # Resumo
    print("\n" + "="*80)
    print("RESUMO DA EXECUÇÃO")
    print("="*80 + "\n")
    
    success_count = sum(1 for r in results if r['status'] == 'SUCCESS')
    failed_count = sum(1 for r in results if r['status'] == 'FAILED')
    
    print(f"Total de arquivos processados: {len(results)}")
    print(f"  ✓ Sucesso: {success_count}")
    print(f"  ✗ Falhas: {failed_count}")
    
    if success_count > 0:
        total_rows = sum(r.get('linhas', 0) for r in results if r['status'] == 'SUCCESS')
        total_size = sum(r.get('tamanho_mb', 0) for r in results if r['status'] == 'SUCCESS')
        print(f"\nTotal de registros processados: {total_rows:,}")
        print(f"Tamanho total em silver: {total_size:.2f} MB")
    
    print("\n" + "="*80 + "\n")
    return results


In [10]:
results_silver = process_bronze_to_silver(bronze_path, silver_path)

CAMO-Net ETL Pipeline - BRONZE → SILVER

Arquivos encontrados: 9


────────────────────────────────────────────────────────────────────────────────
Processando: TAB_ATEND_DIAG
────────────────────────────────────────────────────────────────────────────────
  • Linhas: 574,676
  • Colunas originais: 6

  [1] Padronizando nomes das colunas...
      ✓ 6 colunas renomeadas para snake_case

  [2] Limpando nomes de medicamentos...
      • Nenhuma coluna de medicamento identificada

  [3] Anonimizando dados sensíveis (PII)...
      • Nenhuma coluna PII identificada

  ✓ Salvo em: TAB_ATEND_DIAG.parquet
  • Colunas finais: 6
  • Tamanho: 18.73 MB

────────────────────────────────────────────────────────────────────────────────
Processando: TAB_CIAP_DIAGNOSTICO
────────────────────────────────────────────────────────────────────────────────
  • Linhas: 726
  • Colunas originais: 7

  [1] Padronizando nomes das colunas...
      ✓ 7 colunas renomeadas para snake_case

  [2] Limpando nomes de medi

In [12]:
df_results_silver = pd.DataFrame(results_silver)
df_results_silver


,tabela,status,linhas,colunas_antes,colunas_depois,tamanho_mb
0,TAB_ATEND_DIAG,SUCCESS,574676,6,6,18.727761
1,TAB_CIAP_DIAGNOSTICO,SUCCESS,726,7,7,0.031165
2,TAB_MEDPRESCRITO_ANALISE,SUCCESS,269123,9,9,12.191727
3,TAB_MED_PRESCRITO,SUCCESS,443454,7,7,13.630848
4,TAB_ATENDIMENTO_ANALISE,SUCCESS,298848,13,13,10.942642
5,TAB_ATENDIMENTO,SUCCESS,480439,7,7,19.753783
6,TAB_CID_DIAGNOSTICO,SUCCESS,896,9,9,0.038957
7,TAB_MEDICAMENTO,SUCCESS,33246,12,12,0.633564
8,TAB_UNIDADE_SAUDE,SUCCESS,50,5,5,0.005686


In [13]:
silver_files = sorted(silver_path.glob('*.parquet'))
print(f"Arquivos gerados em silver: {len(silver_files)}")
for f in silver_files:
    print("-", f.name)


Arquivos gerados em silver: 9
- TAB_ATENDIMENTO.parquet
- TAB_ATENDIMENTO_ANALISE.parquet
- TAB_ATEND_DIAG.parquet
- TAB_CIAP_DIAGNOSTICO.parquet
- TAB_CID_DIAGNOSTICO.parquet
- TAB_MEDICAMENTO.parquet
- TAB_MEDPRESCRITO_ANALISE.parquet
- TAB_MED_PRESCRITO.parquet
- TAB_UNIDADE_SAUDE.parquet
